In [14]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from numpy import linalg as ln #to compute eigen values/ vectors
from NetCDF import Dataset

ModuleNotFoundError: No module named 'NetCDF'

In [7]:
print(xr.backends.list_engines())

{'scipy': <ScipyBackendEntrypoint>
  Open netCDF files (.nc, .nc4, .cdf and .gz) using scipy in Xarray
  Learn more at https://docs.xarray.dev/en/stable/generated/xarray.backends.ScipyBackendEntrypoint.html, 'store': <StoreBackendEntrypoint>
  Open AbstractDataStore instances in Xarray
  Learn more at https://docs.xarray.dev/en/stable/generated/xarray.backends.StoreBackendEntrypoint.html}


In [10]:
SST1 = xr.open_dataset("saved_sst_data.nc") #, engine="nc4")

ValueError: found the following matches with the input file in xarray's IO backends: ['netcdf4', 'h5netcdf']. But their dependencies may not be installed, see:
https://docs.xarray.dev/en/stable/user-guide/io.html 
https://docs.xarray.dev/en/stable/getting-started-guide/installing.html

In [12]:
#SST = xr.open_dataset("sst.mon.ltm.1991-2020.nc", engine="nc4")
SST = xr.open_dataset("sst.mon.ltm.1991-2020.nc", engine="scipy")

TypeError: Error: /Users/jessicasmith/Desktop/EAS6995/sst.mon.ltm.1991-2020.nc is not a valid NetCDF 3 file
            If this is a NetCDF4 file, you may need to install the
            netcdf4 library, e.g.,

            $ pip install netcdf4
            